# Proses Pemilihan Model dan Evaluasi 

In [18]:
import pandas as pd
import numpy as np
import pickle
import os # Pastikan ini diimpor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # Pastikan ini diimpor jika belum dari train_model.py
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report, roc_auc_score
)

In [19]:
# Import model-modelnya juga jika belum
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


# === PATH KONFIGURASI ===
# Asumsi script ini dijalankan dari dalam folder 'backend'
# atau sesuaikan path jika dijalankan dari root projek
DATASET_PATH = "data/heart_disease_uci.csv"
MODELS_DIR = "models" # Folder 'models' ada di dalam 'backend'
SCALER_PATH = f"{MODELS_DIR}/heart_disease_scaler.pkl"
LR_OPTIMIZED_MODEL_PATH = f"{MODELS_DIR}/lr_model_optimized.pkl"
RF_OPTIMIZED_MODEL_PATH = f"{MODELS_DIR}/rf_model_optimized.pkl"

print("===== TAHAP 1: MEMUAT DAN MEMPROSES ULANG DATASET (UNTUK KONSISTENSI TEST SET) =====")
try:
    df_raw = pd.read_csv(DATASET_PATH, na_values=["?"])
except FileNotFoundError:
    print(f"Error: File dataset tidak ditemukan di {DATASET_PATH}")
    exit()

===== TAHAP 1: MEMUAT DAN MEMPROSES ULANG DATASET (UNTUK KONSISTENSI TEST SET) =====


In [20]:
# 1. Filter dataset 'Cleveland'
df = df_raw[df_raw["dataset"] == "Cleveland"].copy() # .copy() penting
print(f"Jumlah data setelah filter 'Cleveland': {len(df)}")

Jumlah data setelah filter 'Cleveland': 304


In [21]:
# 2. Drop kolom yang tidak relevan
columns_to_drop = ['id', 'dataset']
df.drop(columns=columns_to_drop, inplace=True)

In [22]:
# 3. Konversi target 'num' menjadi biner
df["target"] = df["num"].apply(lambda x: 1 if x > 0 else 0)
df.drop(columns=["num"], inplace=True)

In [23]:
# 4. Konversi boolean (fbs, exang)
bool_map = {True: 1, False: 0, np.nan: np.nan}
df['fbs'] = df['fbs'].map(bool_map)
df['exang'] = df['exang'].map(bool_map)

In [24]:
# 5. Imputasi nilai hilang (menggunakan assignment langsung, bukan inplace=True pada slice)
print("\nImputasi missing values...")
for col in ["slope", "ca", "thal"]:
    if df[col].isnull().any():
        mode_val = df[col].mode()[0]
        df[col] = df[col].fillna(mode_val) # REVISI: Tidak pakai inplace=True
        print(f"Kolom '{col}' diimputasi dengan modus: {mode_val}")
print("Jumlah missing values setelah imputasi:\n", df.isnull().sum())


Imputasi missing values...
Kolom 'slope' diimputasi dengan modus: upsloping
Kolom 'ca' diimputasi dengan modus: 0.0
Kolom 'thal' diimputasi dengan modus: normal
Jumlah missing values setelah imputasi:
 age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalch      0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [25]:
# 6. Encoding Fitur Kategorikal
df["sex"] = df["sex"].map({"Male": 1, "Female": 0})
categorical_cols = ["cp", "restecg", "slope", "thal"]
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
print(f"\nJumlah kolom setelah get_dummies: {len(df.columns)}")


Jumlah kolom setelah get_dummies: 19


In [26]:
# 7. Pisahkan Fitur (X) dan Target (y)
# Pastikan kolom 'target' ada sebelum di-drop
if 'target' not in df.columns:
    print("Error: Kolom 'target' tidak ditemukan setelah pra-pemrosesan.")
    exit()

X = df.drop(columns=["target"])
y = df["target"]

In [27]:
# 8. Split data untuk mendapatkan X_test_original dan y_test yang konsisten
# Variabel X_train_original dan y_train_original sengaja diberi nama beda
# karena kita tidak akan melatih ulang scaler di sini, hanya menggunakan X_test_original.
_, X_test_original, _, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"\nShape X_test_original: {X_test_original.shape}, Shape y_test: {y_test.shape}")


Shape X_test_original: (61, 18), Shape y_test: (61,)


In [28]:
# === TAHAP 2: MUAT SCALER DAN MODEL YANG SUDAH DILATIH ===
print("\nMemuat scaler dan model...")
try:
    with open(SCALER_PATH, "rb") as f:
        scaler = pickle.load(f)
    with open(LR_OPTIMIZED_MODEL_PATH, "rb") as f:
        lr_optimized_model = pickle.load(f)
    with open(RF_OPTIMIZED_MODEL_PATH, "rb") as f:
        rf_optimized_model = pickle.load(f)
    print("Scaler dan model berhasil dimuat.")
except FileNotFoundError as e:
    print(f"Error saat memuat file model/scaler: {e}")
    print("Pastikan script train_model.py sudah dijalankan dan file .pkl ada di folder 'models'.")
    exit()
except Exception as e:
    print(f"Terjadi error lain saat memuat file: {e}")
    exit()


Memuat scaler dan model...
Scaler dan model berhasil dimuat.


In [29]:
# === TAHAP 3: TRANSFORMASI (SCALING) TEST SET ===
# Pastikan kolom X_test_original SAMA PERSIS dengan yang digunakan saat scaler di-fit
# Cara paling aman adalah menyimpan daftar kolom dari X_train saat scaler di-fit (di train_model.py)
# dan menggunakannya di sini untuk memastikan konsistensi.
# Untuk saat ini, kita asumsikan kolomnya sudah benar karena proses preprocessingnya identik.

# Dapatkan nama kolom dari X_test_original (atau dari X_train yang dipakai untuk fit scaler)
# Ini penting agar DataFrame hasil scaling punya nama kolom yang benar
feature_names = X_test_original.columns

try:
    X_test_scaled_np = scaler.transform(X_test_original)
except ValueError as e:
    print(f"Error saat transform X_test_original: {e}")
    print("Kemungkinan jumlah atau urutan fitur tidak cocok dengan saat scaler di-fit.")
    print(f"Fitur saat scaler di-fit (dari scaler.feature_names_in_ jika tersedia): {getattr(scaler, 'feature_names_in_', 'Tidak tersedia')}")
    print(f"Fitur di X_test_original: {X_test_original.columns.tolist()}")
    exit()

# KONVERSI KEMBALI KE DATAFRAME DENGAN NAMA KOLOM YANG BENAR
X_test_scaled_df = pd.DataFrame(X_test_scaled_np, columns=feature_names, index=X_test_original.index)
print(f"\nShape X_test_scaled_df (setelah scaling dan jadi DataFrame): {X_test_scaled_df.shape}")


Shape X_test_scaled_df (setelah scaling dan jadi DataFrame): (61, 18)


In [30]:
# === TAHAP 4: EVALUASI MODEL ===
models_to_evaluate = {
    "Logistic Regression Optimized": lr_optimized_model,
    "Random Forest Optimized": rf_optimized_model
}

results_summary = {} # Ganti nama variabel 'results' agar tidak konflik jika 'results' nama kolom

print("\n===== HASIL EVALUASI MODEL PADA TEST SET =====")
for model_name, model_object in models_to_evaluate.items():
    print(f"\n--- Model: {model_name} ---")

    # Gunakan DataFrame X_test_scaled_df yang sudah punya nama fitur
    y_pred = model_object.predict(X_test_scaled_df)
    y_prob = model_object.predict_proba(X_test_scaled_df)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    cm = confusion_matrix(y_test, y_pred)

    results_summary[model_name] = { # Ganti nama variabel
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "ROC-AUC": roc_auc,
        "Confusion Matrix": cm
    }

    print(f"Akurasi: {accuracy:.4f}")
    print(f"Presisi: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print("Confusion Matrix:\n", cm)
    print("Classification Report:\n", classification_report(y_test, y_pred))

# === TAHAP 5: PEMILIHAN MODEL TERBAIK BERDASARKAN ROC-AUC ===
print("\n===== PEMILIHAN MODEL TERBAIK =====")
# Cari nama model dengan ROC-AUC tertinggi dari dictionary results_summary
best_model_name = None
best_roc_auc = 0

if results_summary: # Pastikan dictionary tidak kosong
    for name, metrics in results_summary.items():
        print(f"\n{name}: ROC-AUC = {metrics['ROC-AUC']:.4f}")
        if metrics['ROC-AUC'] > best_roc_auc:
            best_roc_auc = metrics['ROC-AUC']
            best_model_name = name
    
    if best_model_name:
        print(f"\nModel terbaik berdasarkan ROC-AUC adalah: {best_model_name} (ROC-AUC: {best_roc_auc:.4f})")
    else:
        print("Tidak dapat menentukan model terbaik dari hasil evaluasi.")
else:
    print("Tidak ada hasil model yang dievaluasi.")


===== HASIL EVALUASI MODEL PADA TEST SET =====

--- Model: Logistic Regression Optimized ---
Akurasi: 0.8852
Presisi: 0.8621
Recall: 0.8929
F1-Score: 0.8772
ROC-AUC: 0.9513
Confusion Matrix:
 [[29  4]
 [ 3 25]]
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.88      0.89        33
           1       0.86      0.89      0.88        28

    accuracy                           0.89        61
   macro avg       0.88      0.89      0.88        61
weighted avg       0.89      0.89      0.89        61


--- Model: Random Forest Optimized ---
Akurasi: 0.8197
Presisi: 0.7742
Recall: 0.8571
F1-Score: 0.8136
ROC-AUC: 0.9383
Confusion Matrix:
 [[26  7]
 [ 4 24]]
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.79      0.83        33
           1       0.77      0.86      0.81        28

    accuracy                           0.82        61
   macro avg       0.82      0.82    